### obsolete
### refer to udf_so_question

In [1]:
import os
import sys

spark_path = "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
#os.environ['PYSPARK_PYTHON'] = 'C:\Users\Wei Zhong\Anaconda3\envs\python2\python' # <---

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sc = SparkContext("local")
sqlCtx = SQLContext(sc)

In [ ]:
# Generate Random Data
import itertools
import random
students = ['John', 'Mike','Matt']
subjects = ['Math', 'Sci', 'Geography', 'History']
random.seed(1)
data = []
 
for (student, subject) in itertools.product(students, subjects):
    data.append((student, subject, random.randint(0, 100)))
 
# Create Schema Object
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
            StructField("student", StringType(), nullable=False),
            StructField("subject", StringType(), nullable=False),
            StructField("score", IntegerType(), nullable=False)
    ])
 
# Create DataFrame 
#from pyspark.sql import HiveContext
#sqlContext = HiveContext(sc)
rdd = sc.parallelize(data)
df = sqlCtx.createDataFrame(rdd, schema)
 
# Define udf
from pyspark.sql.functions import udf
def scoreToCategory(score):
    if score >= 80: return 'A'
    elif score >= 60: return 'B'
    elif score >= 35: return 'C'
    else: return 'D'
 
udfScoreToCategory=udf(scoreToCategory, StringType())
df.withColumn("category", udfScoreToCategory("score")).show(10)

In [11]:
#sc.stop()

In [4]:
df2 = df

In [32]:
from pyspark.sql.functions import exp
df2 = df2.withColumn("score-exp", exp("score"))
df.show()
df2.show()

+-------+---------+-----+
|student|  subject|score|
+-------+---------+-----+
|   John|     Math|   13|
|   John|      Sci|   85|
|   John|Geography|   77|
|   John|  History|   25|
|   Mike|     Math|   50|
|   Mike|      Sci|   45|
|   Mike|Geography|   65|
|   Mike|  History|   79|
|   Matt|     Math|    9|
|   Matt|      Sci|    2|
|   Matt|Geography|   84|
|   Matt|  History|   43|
+-------+---------+-----+

+-------+---------+-----+--------------------+
|student|  subject|score|           score-exp|
+-------+---------+-----+--------------------+
|   John|     Math|   13|   442413.3920089205|
|   John|      Sci|   85|8.223012714622913E36|
|   John|Geography|   77|2.758513454523170...|
|   John|  History|   25|7.200489933738588E10|
|   Mike|     Math|   50|5.184705528587072E21|
|   Mike|      Sci|   45|3.493427105748509...|
|   Mike|Geography|   65|1.694889244410333...|
|   Mike|  History|   79|2.038281066512668...|
|   Matt|     Math|    9|   8103.083927575384|
|   Matt|      Sci|

In [37]:
from pyspark.sql.functions import udf

def foobar(student, df2):
    if student == "Mike":
        print df2.head()
    return "ok"
        
udfScoreToCategory=udf(foobar, StringType())
df.withColumn("ok", udfScoreToCategory("student", df2)).show(10)

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [36]:
print df2.head()

Row(student=u'John', subject=u'Math', score=13, score-exp=442413.3920089205)
